In [1]:
%cd ..

/root/ThinkLogits


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os
import logging
import json

from src.main.pipeline import load_model_and_tokenizer

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_path = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
model, tokenizer, model_name, device = load_model_and_tokenizer(model_path)

2025-04-14 11:57:31,596 - INFO - CUDA is available. Using GPU.
2025-04-14 11:57:31,598 - INFO - Loading model and tokenizer: deepseek-ai/DeepSeek-R1-Distill-Llama-8B onto cuda
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.37s/it]
2025-04-14 11:57:40,110 - INFO - Model and tokenizer loaded successfully.


In [4]:
src.main.generate_dataset_completions(
    model = model,
    tokenizer = tokenizer,
    model_name = model_name,
    device = device,
    hint_types = ["sycophancy"],
    batch_size = 5,
    max_new_tokens = None,
    n_questions = 150
    )

2025-04-14 11:57:40,124 - INFO - Using chat template: <|begin_of_text|><|start_header_id|>user<|end_header_id|>
{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
2025-04-14 11:57:40,126 - INFO - --- Processing dataset for hint type: sycophancy ---
2025-04-14 11:57:40,158 - INFO - Generating completions for sycophancy...
2025-04-14 11:57:40,159 - INFO - Using max_new_tokens: 2048
2025-04-14 11:57:40,160 - INFO - Processing batch 1/30 (Size: 5, QIDs: 0-4)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-04-14 11:58:11,836 - INFO - Processing batch 2/30 (Size: 5, QIDs: 5-9)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-04-14 11:58:33,732 - INFO - Processing batch 3/30 (Size: 5, QIDs: 10-14)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-04-14 11:59:46,902 - INFO - Processing batch 4/30 (Size: 5, QIDs: 15-19)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

In [5]:
src.main.generate_dataset_completions(
    model = model,
    tokenizer = tokenizer,
    model_name = model_name,
    device = device,
    hint_types = ["induced_urgency"],
    batch_size = 5,
    max_new_tokens = None,
    n_questions = 150
    )

2025-04-14 12:19:03,245 - INFO - Using chat template: <|begin_of_text|><|start_header_id|>user<|end_header_id|>
{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
2025-04-14 12:19:03,248 - INFO - --- Processing dataset for hint type: induced_urgency ---
2025-04-14 12:19:03,286 - INFO - Generating completions for induced_urgency...
2025-04-14 12:19:03,287 - INFO - Using max_new_tokens: 2048
2025-04-14 12:19:03,288 - INFO - Processing batch 1/30 (Size: 5, QIDs: 0-4)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-04-14 12:20:16,024 - INFO - Processing batch 2/30 (Size: 5, QIDs: 5-9)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-04-14 12:20:49,435 - INFO - Processing batch 3/30 (Size: 5, QIDs: 10-14)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-04-14 12:21:22,209 - INFO - Processing batch 4/30 (Size: 5, QIDs: 15-19)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end g

In [ ]:
src.main.generate_dataset_completions(
    model = model,
    tokenizer = tokenizer,
    model_name = model_name,
    device = device,
    hint_types = ["unethical_information"],
    batch_size = 5,
    max_new_tokens = None,
    n_questions = 150
    )

2025-04-14 12:38:06,964 - INFO - Using chat template: <|begin_of_text|><|start_header_id|>user<|end_header_id|>
{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
2025-04-14 12:38:06,966 - INFO - --- Processing dataset for hint type: unethical_information ---
2025-04-14 12:38:06,997 - INFO - Generating completions for unethical_information...
2025-04-14 12:38:06,998 - INFO - Using max_new_tokens: 2048
2025-04-14 12:38:06,999 - INFO - Processing batch 1/30 (Size: 5, QIDs: 0-4)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-04-14 12:38:56,166 - INFO - Processing batch 2/30 (Size: 5, QIDs: 5-9)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-04-14 12:39:19,342 - INFO - Processing batch 3/30 (Size: 5, QIDs: 10-14)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2025-04-14 12:40:32,562 - INFO - Processing batch 4/30 (Size: 5, QIDs: 15-19)
Setting `pad_token_id` to `eos_token_id`:128001 fo